## Setup Ollama (requires Docker)

In [ ]:
# Pull the latest ollama Docker image
!docker pull ollama/ollama:latest

In [ ]:
# Run ollama Docker container
!docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

In [ ]:
# Or start it if it's already created
!docker start ollama

In [ ]:
# Pull llama3-chatqa model
!docker exec ollama ollama pull llama3-chatqa

In [ ]:
# Pull nomic-embed-text embeddings model
!docker exec ollama ollama pull nomic-embed-text

## RAG with Ollama

### Install & import dependencies

In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchainhub langchain-chroma langsmith

In [ ]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

### Enable LangSmith tracing (optional)

In [ ]:
# LangSmith tracing
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Demo1"
# os.environ["LANGCHAIN_API_KEY"] = "<Your LangSmith API key>"

### Indexing

You can skip this section if the data is already indexed

In [ ]:
# Load markdown documents from the specified folder
loader = DirectoryLoader('./gitlab_engineering/infrastructure', glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
docs = loader.load()

# Split the documents
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, return_each_line=False, strip_headers=False)

splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]

In [ ]:
# Create an instance of embeddings model integration
embedding = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
# Vectorize and index the split documents
persist_directory="./chroma_db_ollama"
Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=persist_directory)

### Retrieval & generation

In [ ]:
# Create a retriever for the indexed data
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# Pull RAG prompt template. See: https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create an instance of a chat model integration
llm = ChatOllama(model="llama3-chatqa")

# Create a question answering RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Invoke the chain
rag_chain.invoke("What is the manual flow to detect flaky tests?")